# Set up

In [1]:
import io
import pprint
import requests
import json
from keycloak import KeycloakAdmin
from keycloak import KeycloakOpenID
from IPython.display import Image

APP_BASE_URL='http://localhost:8080'
APP_ADMIN_EMAIL='admin@everest.engineering'
APP_ADMIN_PASSWORD='ac0n3x72'

MONITORING_PASSWORD='ac0n3x72'

KEYCLOAK_BASE_URL='http://localhost:8180'
KEYCLOAK_ADMIN_EMAIL='admin@everest.engineering'
KEYCLOAK_ADMIN_PASSWORD='ac0n3x72'

pp = pprint.PrettyPrinter()

default_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}",
    realm_name='default',
    client_id='default-client',
    verify=False)

monitoring_client = KeycloakOpenID(
    server_url=f"{KEYCLOAK_BASE_URL}",
    realm_name='default',
    client_id='monitoring',
    client_secret_key=MONITORING_PASSWORD,
    verify=False)

# User uploads photos

## Simulated user self registration for Bob

In [5]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/",
    username=KEYCLOAK_ADMIN_EMAIL,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [8]:
bob_user_id = keycloak_admin.create_user(
    {
        "username": "bob@example.com",
        "email": "bob@example.com",        
        "enabled": True,
        "attributes": {
            "displayName": "Bob Example" 
        },
        "credentials": [{"value": "password-here", "type": "password"}]
    }
)

pp.pprint(bob_user_id)

'e0949e63-038c-4d7d-aaa1-89d7e243e610'


## Bob logs in

In [9]:
bob_tokens = default_client.token('bob@example.com', 'password-here')
pp.pprint(bob_tokens)

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ3UFljRmtyY1ZocGlwMDVQS2h5eFVBTm9sUHFfRzVaNmVONEQ0M3JycXE0In0.eyJleHAiOjE2NTg4MTgyNDcsImlhdCI6MTY1ODgxNzk0NywianRpIjoiYWUwOGVlMmItNjU3NS00NzhhLWJjYzUtMGQ0ZWE3ZjJlODJiIiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo4MTgwL3JlYWxtcy9kZWZhdWx0Iiwic3ViIjoiZTA5NDllNjMtMDM4Yy00ZDdkLWFhYTEtODlkN2UyNDNlNjEwIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiZGVmYXVsdC1jbGllbnQiLCJzZXNzaW9uX3N0YXRlIjoiODVkZGJhYmYtNTlmOC00OTc0LWFjNzAtYmVmNDNkZDgwNzA1IiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHA6Ly9sb2NhbGhvc3Q6ODA4MCJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiUkVHSVNURVJFRF9VU0VSIl19LCJzY29wZSI6ImRlZmF1bHRfc2NvcGUgcHJvZmlsZSBlbWFpbCIsInNpZCI6Ijg1ZGRiYWJmLTU5ZjgtNDk3NC1hYzcwLWJlZjQzZGQ4MDcwNSIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiZGlzcGxheU5hbWUiOiJCb2IgRXhhbXBsZSIsImdyb3VwcyI6WyIvZGVmYXVsdF9ncm91cCJdLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJib2JAZXhhbXBsZS5jb20iLCJlbWFpbCI6ImJvYkBleGFtcGxlLmNvbSJ9.AM6w9l3wt6vkVHhRqa83lgNUn0TRLfrqdfXbWb1bCN8BZoppx2Jl8AgC2yVOzjxEEnxlSL7O7gf7_EgeBM2EoDKegfkrzDvx0

## Bob uploads some photos

In [12]:
with open('test_photo_1.png', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/photos',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

201


In [13]:
with open('test_photo_2.png', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/photos',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

201


## Application admin login

In [3]:
admin_tokens = default_client.token(APP_ADMIN_EMAIL, APP_ADMIN_PASSWORD)
pp.pprint(admin_tokens)

KeycloakAuthenticationError: 401: b'{"error":"invalid_grant","error_description":"Invalid user credentials"}'

In [ ]:
admin_userinfo = default_client.userinfo(admin_tokens['access_token'])
pp.pprint(admin_userinfo)

# System info

## OID configuration

In [ ]:
pp.pprint(default_client.well_know())

## Application version

In [ ]:
print(requests.get(f'{APP_BASE_URL}/api/version').text)

## Monitoring

In [ ]:
monitoring_tokens = monitoring_client.token(grant_type='client_credentials')
pp.pprint(monitoring_tokens)

In [ ]:
public_health = requests.get(f'{APP_BASE_URL}/actuator/health')
pp.pprint(public_health.json())

In [ ]:
private_health = requests.get(
    f'{APP_BASE_URL}/actuator/health', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(private_health.status_code)
pp.pprint(private_health.json())

## Available system metrics

Most metrics won't show up until later due to lazy loading.

In [ ]:
metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(metrics.status_code)
pp.pprint(metrics.json()['names'])

## Detailed metrics

In [ ]:
request_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/metrics/process.cpu.usage', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
pp.pprint(request_metrics.json())

## Prometheus metrics

Most metrics won't show up until later due ot lazy loading.

In [ ]:
prometheus_metrics = requests.get(
    f'{APP_BASE_URL}/actuator/prometheus', 
    headers=
    {
        "Authorization": f"Bearer {monitoring_tokens['access_token']}"
    }
)
print(prometheus_metrics.status_code)
pp.pprint(prometheus_metrics.text)

## Replay status

In [ ]:
replay_status = requests.get(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
print(replay_status.status_code)
pp.pprint(replay_status.json())

## Trigger replay

Replays when multiple nodes are involved require you to first shut down the tracking event processors so that ownership is released. The node that receives the request to replay will be the one handling it.

This can be done via the dashboard or via `axonserver-cli`.

In [ ]:
response = requests.post(
    f'{APP_BASE_URL}/actuator/replay', 
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    },
    json={}
)
# You are expecting a 204 NO-CONTENT response here
print(response.status_code)

# Self registration scenario

## Simulated user self registration for Bob

In [ ]:
keycloak_admin = KeycloakAdmin(
    server_url=f"{KEYCLOAK_BASE_URL}/",
    username=KEYCLOAK_ADMIN_EMAIL,
    password=KEYCLOAK_ADMIN_PASSWORD,
    realm_name='default',
    user_realm_name='master',
    verify=False)

In [ ]:
bob_user_id = keycloak_admin.create_user(
    {
        "username": "bob@example.com",
        "email": "bob@example.com",        
        "enabled": True,
        "attributes": {
            "displayName": "Bob Example" 
        },
        "credentials": [{"value": "password-here", "type": "password"}]
    }
)

realm_roles = keycloak_admin.get_realm_roles()

for role in realm_roles:
    if role['name'] in ['ORG_ADMIN', 'ORG_USER'] :
        keycloak_admin.assign_realm_roles(user_id=bob_user_id, roles=[role])

pp.pprint(bob_user_id)

## Bob's user and organisation does not exist until first use

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/users',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

In [ ]:
response = requests.get(f'{APP_BASE_URL}/admin/organizations',
    headers=
    {
        "Authorization": f"Bearer {admin_tokens['access_token']}"
    }
)
pp.pprint(response.json())

## Bob logs in

In [ ]:
bob_tokens = default_client.token('bob@example.com', 'password-here')
pp.pprint(bob_tokens)

## Bob retrieves profile details

After this call you can go back up and check - the user and their default organisation should now be available

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/user',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
).json()
pp.pprint(response)
bob_org_id = response['organizationId']

## Bob uploads profile photo

In [ ]:
with open('profile.jpg', 'rb') as image:
    response = requests.post(f'{APP_BASE_URL}/api/user/profile-photo',
        headers=
        {
            "Authorization": f"Bearer {bob_tokens['access_token']}"
        },
        files= 
        {
            'file': image
        }
    )
    print(response.status_code)

## Bob downloads profile photo

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/user/profile-photo/thumbnail?width=1024&height=512',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
Image(data=response.content)

# Bob does stuff

## Bob lists users on his organisation

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/organizations/{bob_org_id}/users',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
pp.pprint(response.json())

## Bob retrieves information about the organisation

In [ ]:
response = requests.get(f'{APP_BASE_URL}/api/organizations/{bob_org_id}',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    }
)
print(response.status_code)
pp.pprint(response.json())

## Bob starts listening for organisation updates

Content negotiation allows you to subscribe to streaming updates. 

This isn't easily demonstrated in this notebook. Run the following to generate the `curl` command that lets you subscribe to streaming updates:

In [ ]:
print(f"curl --verbose -H \"Accept: application/x-ndjson\" -H \"Authorization: Bearer {bob_tokens['access_token']}\" {APP_BASE_URL}/api/organizations/{bob_org_id}")

## Bob changes the organisation's website

In [ ]:
response = requests.put(f'{APP_BASE_URL}/api/organizations/{bob_org_id}',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json = 
    {
        'websiteUrl': 'https://my.nice.example.com/1'
    }
)
print(response.status_code)

## Bob creates Alice, a new user for the organisation

This relies on the realm being configured to require email verification for the new user to be able to log in.

The application will trigger the verification email as part of the set up process.

For this to work you need to configure your realm in Keycloak to:
* send emails via an SMTP host
* require email verfication

(Don't forget to change Alice's email address)

In [ ]:
response = requests.post(f'{APP_BASE_URL}/api/organizations/{bob_org_id}/users',
    headers=
    {
        "Authorization": f"Bearer {bob_tokens['access_token']}"
    },
    json={
        "displayName": "Alice Example",
        "emailAddress": "alice@example.com"
    }
)
print(response.status_code)
print(response.text)
alice_user_id = response.text

## Alice logs into Bob's organisation

In [ ]:
alice_tokens = default_client.token('alice@example.com', 'password-is-set-via-keycloak-email')
pp.pprint(alice_tokens)

# Admin deletes user and discards encryption key

After this call you can go back up and check that replays still work and that Bob's information no longer appears anywhere in the API or event log. 

Note that the access token will still be valid until it expires. You can turn on (very costly) token validation on every request if necessary.

In [ ]:
response = requests.post(f'{APP_BASE_URL}/api/users/{bob_user_id}/forget',
    headers=
    {
        'Authorization': f"Bearer {admin_tokens['access_token']}"
    },
    json={
      'requestReason': 'GDPR request received'
    }                  
)
print(response)